In [ ]:
# Import Packages
from os.path import normpath, join
import numpy as np
import scipy.io as sio
import scipy as sc
import numpy as np
import math
import matplotlib.pyplot as plt
from skimage import io
import cv2 as cv2
import seaborn as sns
import matplotlib

In [ ]:
# Read in flow data from 'MATLAB PIV packages'
path2file = "/Users/.../"
flowFile = "Movie1_External_Flow_Field.mat"

# define filepath
fileNameFlow = normpath (join (path2file, flowFile))
# # import file into a dictionary
flowData = sio.loadmat (fileNameFlow)

# read in the matlab flow structure field(idx).{vx|vy}
flowStruct = flowData ["vfilt"]

im_cb = io.imread('/Users/.../cb1_m.tif') # load cell image (this will be a .tif stack)

index = 0 

vx = flowStruct[0,index]['vx'] # get the fields (x components)
vy = flowStruct[0,index]['vy'] # get the fields (y components)
y, x = np.mgrid[0:vx[index].shape[0], 0:vx[index].shape[0]]

vmag = np.sqrt(vx**2 + vy**2)
interval = (slice(None, None, 10), slice(None, None, 10))

dx = 10
dy = 10

img1 = im_cb[index]

## Display the direction of the principal strain with ellipses ##

In [ ]:
phi = np.linspace(0, 2*np.pi, 50)
cosphi = np.cos(phi)
sinphi = np.sin(phi)

plt.imshow(img1, "gray", origin='upper')         
ax1.set_axis_off()

for ii in range(0,vx.shape[0]-dy, dy):
    for jj in range(0,vx.shape[0]-dx, dx):
        
        dudx = (vx[ii, jj + dx] - vx[ii, jj - dx]) # / 2 * dx
        dvdy = (vy[ii + dy, jj] - vy[ii - dy, jj]) # / 2 * dy
        
        L = np.array([[dudx/2*dx, dudx/2*dx], [dvdy/2*dx, dvdy/2*dx]])
        strainRateTensor = (L + L.T)/2
        d,V = sc.linalg.eigh(strainRateTensor)
        
        if np.abs(d[0]) > 0 and np.abs(d[1]) > 0 and np.abs(d[0]) != np.abs(d[1]):
            pc1 = np.max(np.abs(d))
                
            ind1 = [i for (i, val) in enumerate(np.abs(d)) if val == pc1]
            pc1a = d[ind1[0]]*.5

            ind2 = [i for (i, val) in enumerate(np.abs(d)) if val != pc1]           
            pc2 = d[ind2[0]]*.5

            pa1 = math.atan2(V[1,0],V[0,0])            
#             pa2 = math.atan2(V[1,1],V[0,1])
            
            xy = np.array([cosphi*pc1a, sinphi*pc2])
            
            theta = pa1
            c, s = np.cos(pa1), np.sin(pa1)            
            R = np.array(((c,-s), (s, c)))
            
            xx,yy = R.dot(xy)  

#           pc1R = [i * R for i in np.cos(t)] 
#           pc2R = [i * R for i in np.sin(t)] 

            if pc1a < 0:
                plt.plot(x[ii,jj]+xx, y[ii,jj]+yy, color = sns.xkcd_rgb["neon blue"],linewidth=0.1)
            
#             or use patches to make the ellipses
#                 ell1 = matplotlib.patches.Ellipse([x[ii,jj], y[ii,jj]],
#                                                  pc1a, pc2, angle=np.rad2deg(pa1),
#                                                  color = sns.xkcd_rgb["neon blue"]) or use patches
            else:
                plt.plot(x[ii,jj]+xx, y[ii,jj]+yy, color = sns.xkcd_rgb["neon pink"],linewidth=0.1)
# plt.savefig('/Users/../../flows001{}.pdf'.format(index), bbox_inches='tight', pad_inches=0, dpi=600)

plt.show()
                     